<a href="https://colab.research.google.com/github/nourwalid70/Modulation-Classification/blob/main/Lab_4_Modulation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub
!pip install --upgrade tensorflow
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2


In [ ]:
import gc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Conv2D, Conv1D 
from tensorflow.keras.layers import MaxPool2D, MaxPool1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.optimizers import Adam
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import glob
import librosa
from librosa import display
import numpy as np
import matplotlib.pyplot as plt
import pickle as cPickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import torch
import torch.nn as nn
import torch.nn.functional
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
from torchsummary import summary
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from scipy import integrate
from keras.models import load_model

In [ ]:
from keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

#Data Generation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# differentiation and integration features
def add_features(data):
  new_data = []
  #np.empty((0, 3*data.shape[1], data.shape[2]))
  for sample in data:
    # differentiation
    #grad = np.gradient(sample, axis = 1)
    
    # integration
    intg = integrate.cumtrapz(sample, initial=0)

    # append data
    new_sample = np.vstack((sample, intg))
    new_data.append(new_sample)

  return np.array(new_data)

In [ ]:
x = []  
labels = []

In [ ]:
# reading data
path = "/content/drive/MyDrive/Colab Notebooks/PR lab4/RML2016.10b.dat"
f = open(path,'rb')
df = pd.read_pickle(path)
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], df.keys())))), [1,0])

for mod in mods:
    for snr in snrs:
        x.append(df[(mod,snr)])
        for i in range(df[(mod,snr)].shape[0]):  labels.append((mod,snr))
x = np.vstack(x)
labels=np.array(labels)

In [ ]:
# adding the new features
print(x.shape)
x = add_features(x)
print(x.shape)

In [ ]:
#split data
train_data, test_data, train_labels,test_labels = train_test_split(x,labels,test_size=0.3, random_state = 10, stratify = labels)
train_data, val_data, train_labels,val_labels = train_test_split(train_data,train_labels, test_size=0.05,random_state=12, stratify=train_labels)

In [ ]:
#keep old test labels for big picture
old_test_labels = test_labels

In [ ]:
# encode labels [0, .. ,9]
label = LabelEncoder() 
train_labels= label.fit_transform(train_labels[:,0])
test_labels= label.fit_transform(test_labels[:,0])
val_labels= label.fit_transform(val_labels[:,0])

In [ ]:
# clear unnecessary memory
del df
del x
gc.collect()

# CNN

In [ ]:
train_data=train_data.reshape((train_data.shape[0],1,train_data.shape[2],train_data.shape[1]))
test_data=test_data.reshape((test_data.shape[0],1,test_data.shape[2],test_data.shape[1]))
val_data=val_data.reshape((val_data.shape[0],1,val_data.shape[2],val_data.shape[1]))

In [ ]:
train_data.shape, val_data.shape, test_data.shape

((798000, 1, 128, 2), (42000, 1, 128, 2), (360000, 1, 128, 2))

In [ ]:
train_labels = to_categorical(list(train_labels))
test_labels = to_categorical(list(test_labels))
val_labels = to_categorical(list(val_labels))

In [ ]:
cnn=Sequential()
cnn.add(Conv2D(64,(1,3),1,activation='relu',padding='same',kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.001),input_shape=(train_data.shape[1],train_data.shape[2],train_data.shape[3])))
cnn.add(Conv2D(16,(2,3),1,activation='relu',padding='same',kernel_regularizer=tensorflow.keras.regularizers.l2(l=0.001)))
cnn.add(Flatten())
cnn.add(Dense(128,activation='relu'))
cnn.add(Dense(10,activation='softmax'))

In [ ]:
cnn.compile(loss='categorical_crossentropy',optimizer=tensorflow.keras.optimizers.Adam(0.0005,),metrics=['accuracy'])
cnn.fit(train_data,train_labels,batch_size=64,epochs=100,validation_data=(val_data,val_labels))

#Vanella RNN

In [ ]:
# to fit in RNN input
train_data = np.transpose(train_data, (0, 2, 1))
test_data = np.transpose(test_data, (0, 2, 1))
val_data = np.transpose(val_data, (0, 2, 1))

In [ ]:
# feature minMax normalization
train_data = train_data.reshape(len(train_data),-1)
test_data = test_data.reshape(len(test_data),-1)
val_data = val_data.reshape(len(val_data),-1)

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)
val_data = scaler.transform(val_data)

train_data = train_data.reshape(len(train_data),128,4)
test_data = test_data.reshape(len(test_data),128,4)
val_data = val_data.reshape(len(val_data),128,4)

In [ ]:
train_data.shape, test_data.shape, val_data.shape

((798000, 128, 4), (360000, 128, 4), (42000, 128, 4))

In [ ]:
def RNN_model():
  rnn=Sequential()
  rnn.add(SimpleRNN(100,input_shape=(128,4),return_sequences=True))
  rnn.add(SimpleRNN(100,return_sequences=True))
  rnn.add(SimpleRNN(50))
  rnn.add(Dense(10,activation='softmax'))
  return rnn

In [ ]:
# training
callback = EarlyStopping(monitor='loss', patience=3)
rnn = RNN_model()
rnn.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

In [ ]:
#load model
rnn = load_model('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained5.h5')

In [ ]:
#save model
rnn.save('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained5.h5')

In [ ]:
#train for one epoch
callback = EarlyStopping(monitor='loss', patience=3)
for i in range(100):
  rnn.fit(train_data,train_labels,batch_size=64,epochs=1,validation_data=(val_data,val_labels), callbacks=[callback])
  #save model
  rnn.save('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained5.h5')

#LSTM



In [ ]:
# to fit in LSTM input
train_data = np.transpose(train_data, (0, 2, 1))
test_data = np.transpose(test_data, (0, 2, 1))
val_data = np.transpose(val_data, (0, 2, 1))

In [ ]:
# feature minMax normalization
train_data = train_data.reshape(len(train_data),-1)
test_data = test_data.reshape(len(test_data),-1)
val_data = val_data.reshape(len(val_data),-1)

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)
val_data = scaler.transform(val_data)

train_data = train_data.reshape(len(train_data),128,4)
test_data = test_data.reshape(len(test_data),128,4)
val_data = val_data.reshape(len(val_data),128,4)

In [ ]:
train_data.shape, test_data.shape, val_data.shape

((798000, 128, 4), (360000, 128, 4), (42000, 128, 4))

In [ ]:
def LSTM_model():
  model = Sequential()
  model.add(LSTM(100, return_sequences=True, unroll=True, input_shape=(128,4))) # 3D
  model.add(LSTM(100, return_sequences=True, unroll=True))
  model.add(LSTM(50, unroll=True))
  model.add(Dense(10,activation='softmax'))
  return model

In [ ]:
# training
callback = EarlyStopping(monitor='loss', patience=3)
lstm = LSTM_model()
lstm.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.001, decay=1e-6), metrics=['accuracy'])

In [ ]:
# save model
lstm.save('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained14.h5')

In [ ]:
#load model
lstm = load_model('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained14.h5')

In [ ]:
lstm.summary()

In [ ]:
#train for one epoch
callback = EarlyStopping(monitor='loss', patience=3)
for i in range(100):
  lstm.fit(train_data,train_labels,batch_size=64,epochs=1,validation_data=(val_data,val_labels), callbacks=[callback])
  #save model
  lstm.save('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained14.h5')

# Bonus - CNN-LSTM


In [ ]:
# to fit in LSTM input
train_data = np.transpose(train_data, (0, 2, 1))
test_data = np.transpose(test_data, (0, 2, 1))
val_data = np.transpose(val_data, (0, 2, 1))

In [ ]:
# feature minMax normalization
train_data = train_data.reshape(len(train_data),-1)
test_data = test_data.reshape(len(test_data),-1)
val_data = val_data.reshape(len(val_data),-1)

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)
val_data = scaler.transform(val_data)

train_data = train_data.reshape(len(train_data),128,4)
test_data = test_data.reshape(len(test_data),128,4)
val_data = val_data.reshape(len(val_data),128,4)

In [ ]:
train_data=train_data.reshape((train_data.shape[0],1,train_data.shape[1],train_data.shape[2]))
test_data=test_data.reshape((test_data.shape[0],1,test_data.shape[1],test_data.shape[2]))
val_data=val_data.reshape((val_data.shape[0],1,val_data.shape[1],val_data.shape[2]))

In [ ]:
train_data.shape

(798000, 1, 128, 4)

In [ ]:
def cnn_lstm_model():
  cnn_lstm_model = Sequential()
  cnn_lstm_model.add(TimeDistributed(Conv1D(filters=64,kernel_size=4,activation='relu'),input_shape=(1,128,4)))
  cnn_lstm_model.add(TimeDistributed(MaxPool1D(pool_size= 2, strides=2)))
  cnn_lstm_model.add(TimeDistributed(Flatten()))
  cnn_lstm_model.add(LSTM(32, dropout = 0.3, recurrent_dropout = 0.3))
  cnn_lstm_model.add(Dense(10, activation = 'sigmoid'))
  return cnn_lstm_model

In [ ]:
# training
callback = EarlyStopping(monitor='loss', patience=3)
cnn_lstm = cnn_lstm_model()
cnn_lstm.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001, decay=1e-6), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
cnn_lstm = load_model('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained43.h5')

In [ ]:
#train for one epoch
callback = EarlyStopping(monitor='loss', patience=3)
for i in range(100):
  cnn_lstm.fit(train_data,train_labels,batch_size=64,epochs=1,validation_data=(val_data,val_labels), callbacks=[callback])
  #save model
  cnn_lstm.save('/content/drive/MyDrive/Colab Notebooks/PR lab4/partly_trained43.h5')

# Big picture

In [ ]:
def overall_test(model, test_data, test_labels):
    # overall test results
    h, overall_acc = model.evaluate(test_data, test_labels)
    print('overall_acc : ', overall_acc)

    
    y_prob = model.predict(test_data) 
    y_pred = y_prob.argmax(axis=-1)

    overall_c = confusion_matrix(test_labels, y_pred)
    print('overall_confusion :\n', overall_c)

    return overall_acc, overall_c

In [ ]:
# big picture
def big_picture(model, old_test_labels, test_data, test_labels):
  #sort 
  sort_indices = old_test_labels[:, 1].astype('int').argsort()
  old_test_labels = old_test_labels[sort_indices]
  test_data = test_data[sort_indices]
  test_labels = test_labels[sort_indices]

  #split
  temp = old_test_labels[:,1].astype('float64')
  split_indices = np.where(np.diff(temp))[0]+1
  splitted_test_data = np.split(test_data, split_indices)
  splitted_test_labels = np.split(test_labels, split_indices)
  
  # test results per snr
  snr = np.unique(old_test_labels[:,1].astype('int')).tolist()
  acc = []
  conf = []
  for data, labels in zip(splitted_test_data, splitted_test_labels):
    h, a = model.evaluate(data, labels)
    acc.append(a)

    y_prob = model.predict(data) 
    y_pred = y_prob.argmax(axis=-1)

    c = confusion_matrix(labels, y_pred)
    conf.append(c)

  return snr, acc, conf 

In [ ]:
def plot_conf(cf_matrix):
  labels = ["8PSK","AM-DSB","BPSK","CPFSK","GFSK","PAM4","QAM16","QAM64","QPSK","WBFM"]
  plt.rcParams["figure.figsize"] = (15,15)

  ax = sns.heatmap(cf_matrix, annot=True, fmt='d', cmap='Blues')

  ax.set_title('Confusion Matrix\n\n');
  ax.set_xlabel('\nPredicted')
  ax.set_ylabel('Actual');

  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

  plt.show()

In [ ]:
def plot_snr(snr, acc):
  plt.rcParams["figure.figsize"] = (10,10)
  plt.plot(snr, acc)
  plt.xlabel('snr')
  plt.ylabel('accuracy')
  plt.grid(color = 'lime', linestyle = '--', linewidth = 0.5)
  plt.show()

In [ ]:
overall_acc, overall_c = overall_test(cnn_lstm, test_data, test_labels)

In [ ]:
snr, acc, conf  = big_picture(cnn_lstm, old_test_labels, test_data, test_labels)

In [ ]:
plot_conf(overall_c)

In [ ]:
plot_snr(snr, acc)

In [ ]:
snr, acc, conf